In [65]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

device = torch.device("cuda")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)

In [66]:
import json
from torch.utils.data import Dataset, DataLoader

class JsonDataset(Dataset):
    def __init__(self, json_file):
        with open(json_file, 'r') as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data[idx]["Text_inputs"]
        output_text = self.data[idx]["Outputs"]
        return input_text, output_text

inputs = tokenizer(
    "Move Up by 25 cm",
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=512,
).to(model.device)

generated_ids = model.generate(
    **inputs,
    max_length=512,
    num_beams=4,
    early_stopping=True,
)

output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(output_text)

BATCH_SIZE = 64

dataset = JsonDataset("robot_movements_3000.json")
train_data = DataLoader(dataset, BATCH_SIZE, shuffle=True)

optimizer = optim.AdamW(model.parameters(), lr=5e-5)

lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

Move Up by 25 cm


In [67]:
def train(model, tokenizer, train_data, device, epochs=20, lr=5e-5):
    model.to(device)
    model.train()

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    for epoch in range(epochs):
        total_loss = 0

        for input_texts, output_texts in train_data:
            # Tokenize inputs
            inputs = tokenizer(
                list(input_texts),
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=100
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}

            # Tokenize labels
            labels = tokenizer(
                list(output_texts),
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=70
            ).input_ids.to(device)
            labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in loss

            # Forward pass with internal loss
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss


            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_data)
        print(f"Epoch {epoch + 1}, Avg Loss: {avg_loss:.4f}")

def evaluate(model, tokenizer, input_text, device):
    model.eval()
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True, truncation=False, max_length = 512)

In [68]:
# train(model, tokenizer, train_data, device)

In [69]:
# torch.save(model.state_dict(), "BART.pth")

In [70]:
def evaluate(model, tokenizer, input_text, device):
    model.eval()
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)

    output = tokenizer.decode(outputs[0], skip_special_tokens=False)
    print(output)

    return tokenizer.decode(outputs[0], skip_special_tokens=True, truncation=False)

In [71]:
model.load_state_dict(torch.load("BART.pth"))

input_text = "Move forward by 25 then right by 15 and up by 16"
output_text = evaluate(model, tokenizer, input_text, device)
print(f"Input: {input_text}")
print(f"Output: {output_text}")

C:\Users\MSI-Admin\AppData\Local\Temp\ipykernel_26996\184502733.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("BART.pth"))


</s><s>Y += 25, X += 15, Z += 16, End Task</s>
Input: Move forward by 25 then right by 15 and up by 16
Output: Y += 25, X += 15, Z += 16, End Task
